In [1]:
# Tensorflow to open-source’owy framework stworzony przez Google’a do obliczeń numerycznych. 
# Oferuje on zestaw narzędzi służących do projektowania, trenowania oraz douczania (ang. fine-tuning) sieci neuronowych.
# TensorBoard to narzędzie służące do wizualizacji danych. Pozwala prezentować w czytelny sposób m.in. grafy obliczeniowe 
# sieci neuronowych (ang. computational graph), dane wejściowe, dane wyjściowe czy postęp treningu. Dzięki temu ułatwia 
# ono znajdowanie błędów w architekturze oraz optymalizację sieci neuronowych.

In [2]:
# Software requirements
# NVIDIA® GPU drivers —CUDA 10.1 requires 418.x or higher.
# CUDA® Toolkit —TensorFlow supports CUDA 10.1 (TensorFlow >= 2.1.0)
# CUPTI ships with the CUDA Toolkit.
# cuDNN SDK (>= 7.6)
#(Optional) TensorRT 6.0 to improve latency and throughput for inference on some models.

In [3]:
# Source: Python for Data Science and Machine Learning Bootcamp, kurs z platformy Udemy, Stworzony przez Jose Portilla
# Ostatnia aktualizacja: 5/2020

In [4]:
import pandas as pd
import numpy as np
import os
import sqlite3

In [5]:
os.chdir(r"C:\Users\Mateusz\Desktop\WSBProjekt")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Mateusz\\Desktop\\WSBProjekt'

In [ ]:
conn = sqlite3.connect('data\predictions.db')
c = conn.cursor()
indicator="SP500"
# SELECT QUERY dla indexu SP500
df = pd.read_sql_query("""select x."Date", co."Index" as Crude_Oil, 
cr."Index" as Copper, s."Index" as Silver, p."Index" as Platinum,
ip."Index" as Industrial_Production, pa."Index" as Palladium, x."Index" as """+indicator+"""
 from """+indicator+""" as x
  LEFT OUTER JOIN Crude_Oil as co ON x.Date=co.Date
  LEFT OUTER JOIN Copper as cr ON x.Date=cr.Date
  LEFT OUTER JOIN Silver as s ON x.Date=s.Date
  LEFT OUTER JOIN Platinum as p ON x.Date=p.Date
  LEFT OUTER JOIN Industrial_Prod as ip ON strftime('%Y-%m', x.Date)=strftime('%Y-%m', ip.Date)
  LEFT OUTER JOIN Palladium as pa ON x.Date=pa.Date
Where x."Index" and co."Index" and cr."Index" and s."Index" and p."Index"
and ip."Index" and pa."Index" IS NOT NULL
and x.Date < '2020-04-01'
""", conn)

In [ ]:
df

In [ ]:
# Ze względu na to że dane trainowe mają mieć wartości float, usuwam kolumne daty
df = df.drop('Date',axis=1)

In [ ]:
df

In [ ]:
# Training And Test Data
X = df.drop('SP500',axis=1).values
y = df['SP500'].values

In [ ]:
y

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Dzielenie tablic na losowe podzbiory trainowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=101)

In [ ]:
# Scalling data
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Creating model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
from datetime import datetime

In [ ]:
datetime.now().strftime("%Y-%m-%d--%H%M")

In [ ]:
# WINDOWS: Use "logs\\fit"
# MACOS/LINUX: Use "logs\fit"

log_directory = 'logs\\fit'

board = TensorBoard(log_dir=log_directory,histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=1)

In [ ]:
# Teraz utwórzę warstwy modelu
model = Sequential()
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=15,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
# Train the model
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop,board]
          )

In [ ]:
print(log_directory)

In [ ]:
pwd

### Użyj cd w wierszu poleceń, aby zmienić katalog na ścieżkę pliku zgłoszoną przez pwd lub bieżącą lokalizację pliku .py
### Następnie uruchom ten kod w wierszu polecenia lub terminalu, wpisując - 'tensorboard --logdir logs\fit'
### Tensorboard będzie działał lokalnie w twojej przeglądarce pod adresem [http://localhost:6006/](http://localhost:6006/)


In [ ]:
# Możemy użyć interfejsów API tensorboard.notebook, aby uzyskać nieco większą kontrolę:
from tensorboard import notebook
notebook.list()